<a href="https://colab.research.google.com/github/jaqueantunes/dados_saude/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#app

In [9]:
# --- BLOCO 1: INSTALAÇÃO DO NGROK ---
import os

# 1. Instala as bibliotecas
print("⏳ Instalando bibliotecas e Ngrok...")
!pip install -q streamlit pandas plotly pyngrok

from pyngrok import ngrok

# 2. Configura a sua chave (Token)
# SUBSTITUA O TEXTO ABAIXO PELO SEU TOKEN DO SITE
meu_token = "37r0T9pchX0Ky296GxG7j7M9lbC_28kPG8S9eE2Bd4cpxUtpo"

# Autentica
ngrok.set_auth_token(meu_token)
print("✅ Ngrok configurado com sucesso!")

⏳ Instalando bibliotecas e Ngrok...
✅ Ngrok configurado com sucesso!


In [12]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# 1. Configuração da página
st.set_page_config(page_title="Dashboard Obesidade", layout="wide")
st.title("📊 Análise de Dados: Obesidade")

# URL do GitHub
url = "https://raw.githubusercontent.com/jaqueantunes/dados_saude/main/Obesity.csv"

# --- DICIONÁRIO DE TRADUÇÃO ---
# Aqui definimos como vamos chamar cada coluna confusa
dicionario_colunas = {
    'Gender': 'Gênero',
    'Age': 'Idade',
    'Height': 'Altura',
    'Weight': 'Peso',
    'family_history_with_overweight': 'Histórico Familiar Obesidade',
    'FAVC': 'Consumo Alta Caloria (FAVC)',
    'FCVC': 'Consumo Vegetais (FCVC)',
    'NCP': 'Refeições Principais (NCP)',
    'CAEC': 'Comida entre Refeições (CAEC)',
    'SMOKE': 'Fumante',
    'CH2O': 'Consumo Água (CH2O)',
    'SCC': 'Monitora Calorias',
    'FAF': 'Freq. Atividade Física (FAF)',
    'TUE': 'Tempo em Telas (TUE)',
    'CALC': 'Consumo Álcool',
    'MTRANS': 'Transporte',
    'NObeyesdad': 'Nível de Obesidade'
}

@st.cache_data
def carregar_dados():
    df = pd.read_csv(url)
    # AQUI A MÁGICA ACONTECE: Renomeia as colunas usando o dicionário
    df = df.rename(columns=dicionario_colunas)
    return df

try:
    df = carregar_dados()

    # Criamos as abas
    aba1, aba2 = st.tabs(["📋 Base de Dados & Dicionário", "📈 Análise Gráfica"])

    # --- ABA 1: DADOS E GLOSSÁRIO ---
    with aba1:
        st.header("Visualização da Tabela")

        # Botão para mostrar o significado das siglas (Sugestão de melhoria!)
        with st.expander("ℹ️ Entenda as siglas (Glossário)"):
            st.markdown("""
            * **FAVC**: Consumo frequente de alimentos calóricos
            * **FCVC**: Frequência de consumo de vegetais
            * **NCP**: Número de refeições principais
            * **CAEC**: Consumo de comida entre as refeições
            * **CH2O**: Consumo diário de água
            * **FAF**: Frequência de atividade física
            * **TUE**: Tempo de uso de dispositivos tecnológicos
            """)

        col1, col2 = st.columns(2)
        col1.metric("Total de Pacientes", df.shape[0])
        col2.metric("Variáveis Analisadas", df.shape[1])

        st.dataframe(df)

    # --- ABA 2: GRÁFICOS ---
    with aba2:
        st.header("Exploração Visual")
        st.write("Selecione as variáveis (agora em português) para analisar:")

        # Identifica colunas numéricas (elas já estarão com nomes traduzidos)
        numeric_cols = df.select_dtypes(include=['number']).columns
        # Identifica colunas de texto (categóricas)
        cat_cols = df.select_dtypes(include=['object']).columns

        if len(numeric_cols) > 0:
            c1, c2, c3 = st.columns(3)

            with c1:
                col_x = st.selectbox("Eixo X (Horizontal):", numeric_cols, index=0)
            with c2:
                # Tenta achar 'Nível de Obesidade' para ser a cor padrão, senão pega a primeira opção
                index_cor = 0
                if 'Nível de Obesidade' in cat_cols:
                    index_cor = list(cat_cols).index('Nível de Obesidade')
                col_cor = st.selectbox("Agrupar por cor:", cat_cols, index=index_cor)
            with c3:
                 tipo_grafico = st.selectbox("Tipo de Gráfico:", ["Histograma", "Boxplot", "Violino"])

            # Lógica dos Gráficos
            if tipo_grafico == "Histograma":
                fig = px.histogram(df, x=col_x, color=col_cor, barmode='group', title=f"Distribuição de {col_x} por {col_cor}")
            elif tipo_grafico == "Boxplot":
                fig = px.box(df, x=col_cor, y=col_x, color=col_cor, title=f"{col_x} por {col_cor}")
            elif tipo_grafico == "Violino":
                 fig = px.violin(df, x=col_cor, y=col_x, color=col_cor, box=True, title=f"Densidade de {col_x} por {col_cor}")

            st.plotly_chart(fig, use_container_width=True)
        else:
            st.warning("Não há colunas numéricas suficientes.")

except Exception as e:
    st.error(f"Erro ao carregar dados: {e}")

Overwriting app.py


In [ ]:
# --- BLOCO 3: INICIAR SERVIDOR ---
import time

# Mata processos antigos para evitar conflito
!pkill streamlit

# Roda o Streamlit em segundo plano
print("🚀 Iniciando Streamlit...")
get_ipython().system_raw('streamlit run app.py &')

# Aguarda um pouco para o Streamlit iniciar
time.sleep(3)

# Cria o túnel
print("🔗 Conectando ao Ngrok...")
try:
    # Fecha túneis antigos se houver
    ngrok.kill()

    # Abre o novo túnel na porta 8501
    url_publica = ngrok.connect(8501).public_url

    print("\n" + "="*40)
    print(f"👉 SEU NOVO LINK RÁPIDO: {url_publica}")
    print("="*40 + "\n")
    print("(Clique no link acima. Se aparecer um aviso, clique em 'Visit Site')")

    # Mantém o código rodando
    while True:
        time.sleep(1)

except Exception as e:
    print(f"Erro: {e}")

🚀 Iniciando Streamlit...
🔗 Conectando ao Ngrok...

👉 SEU NOVO LINK RÁPIDO: https://nonequalized-subhemispherically-fausto.ngrok-free.dev

(Clique no link acima. Se aparecer um aviso, clique em 'Visit Site')
